In [56]:
import pandas as pd
import numpy as np

In [57]:
def enter_trade(n, prices_dataframe, Lambda):
    # calculate alpha ratio
    prices_dataframe['alpha'] = prices_dataframe['ko_Close'] / prices_dataframe['pep_Close']

    # calculate z_score
    def zscore(n):
        df_mean = prices_dataframe['alpha'].rolling(n, center=False).mean().shift(1)
        df_std = prices_dataframe['alpha'].rolling(n, center=False).std(ddof=1).shift(1)
        z_score = (prices_dataframe['alpha'] - df_mean) / df_std
        return z_score

    z_score = zscore(n)
    prices_dataframe['z_score'] = z_score

    # determine entry status for Y(ko)
    conditions = [
        (prices_dataframe['z_score'] > Lambda),
        (prices_dataframe['z_score'] <= Lambda) & (prices_dataframe['z_score'] >= -Lambda),
        (prices_dataframe['z_score'] < -Lambda)
    ]
    values = ['sell', 'do not enter', 'buy']
    prices_dataframe['action_y'] = np.select(conditions, values)
    prices_dataframe['date'] = prices_dataframe['Date'].shift(-1)
    prices_dataframe['price_y'] = prices_dataframe['ko_Open'].shift(-1)
    prices_dataframe['amount_y'] = 100  #####default enter amount is 100
    prices_dataframe['ticker_y'] = 'ko'

    # determine entry status for X(pep)
    conditions = [
        (prices_dataframe['action_y'] == 'sell'),
        (prices_dataframe['action_y'] == 'do not enter'),
        (prices_dataframe['action_y'] == 'buy')
    ]
    values = ['buy', 'do not enter', 'sell']
    prices_dataframe['action_x'] = np.select(conditions, values)
    prices_dataframe['price_x'] = prices_dataframe['pep_Open'].shift(-1)
    prices_dataframe['amount_x'] = prices_dataframe["price_y"] * prices_dataframe['amount_y'] / prices_dataframe[
        'price_x']
    prices_dataframe['ticker_x'] = 'pep'

    # create dataframe for Y(ko)
    df1 = prices_dataframe[['date', 'ticker_y', 'price_y', 'amount_y', 'action_y']]
    df_y = df1[(df1.action_y != '0') & (df1.action_y != 'do not enter')]
    df_y.rename(columns={'ticker_y': 'ticker', 'price_y': 'price', 'amount_y': 'quantity', 'action_y': 'action'},
                inplace=True)

    # create dataframe for X(pep)
    df2 = prices_dataframe[['date', 'ticker_x', 'price_x', 'amount_x', 'action_x']]
    df_x = df2[(df2.action_x != '0') & (df2.action_x != 'do not enter')]
    df_x.rename(columns={'ticker_x': 'ticker', 'price_x': 'price', 'amount_x': 'quantity', 'action_x': 'action'},
                inplace=True)

    # concat X and Y
    frames = [df_y, df_x]
    result = pd.concat(frames)
    result['trip'] = 'entry'
    result['status'] = 'FILLED'
    result['date'] = pd.to_datetime(result['date'])
    result.sort_values(by='date', inplace=True, ascending=True)
    return result

In [58]:
def exit_trade(n, prices_dataframe, entry_trade, rho, L, X):
    # copy entry_trade 作为exit修改的基础

    first_index = entry_trade.first_valid_index()
    last_index = entry_trade.index[-1]

    exit_order = entry_trade.copy(deep=True)

    # 遍历 entry_trade 修改action和trip的状态
    exit_order['trip'] = 'exit'

    # change LMT price
    price2 = [
        (exit_order['action'] == 'sell'),
        (exit_order['action'] == 'buy')
    ]
    # create a list of values we want to assign for each condition
    price2_values = [exit_order['price'] * (1 + rho), exit_order['price'] * (1 - rho)]
    # create a new column and use np.select to assign values to it using our lists as arguments
    exit_order['price_exit'] = np.select(price2, price2_values)

    # create a list of action2 to change action
    action2 = [
        (exit_order['action'] == 'sell'),
        (exit_order['action'] == 'buy')
    ]
    # create a list of values we want to assign for each condition
    action2_values = ['buy', 'sell']
    # create a new column and use np.select to assign values to it using our lists as arguments
    exit_order['action_exit'] = np.select(action2, action2_values)

    del exit_order['action'], exit_order['price']
    order = ['date', 'ticker', 'price_exit', 'quantity', 'action_exit', 'trip', 'status']
    exit_order = exit_order[order]
    exit_order.columns = ['date', 'ticker', 'price', 'quantity', 'action', 'trip', 'status']

    # 判断status
    exit_order['date_exit'] = ''
    exit_order['status_exit'] = ''

    for i in range(first_index, last_index + 1):
        if i in exit_order.index:
            if exit_order['ticker'][i].values[0] == 'ko' and exit_order['action'][i].values[0] == 'buy':
                for j in range(i + 1, i + X + 1):  # x=60
                    Loss = ((prices_dataframe['ko_Close'][j] - exit_order['price'][i].values[0]) /
                            exit_order['price'][i].values[0]) * 100
                    if Loss > L:  # L=20
                        exit_order['status_exit'][i] = 'STOPLOSS'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    elif exit_order['price'][i].values[0] > prices_dataframe['ko_Close'][j]:
                        exit_order['status_exit'][i] = 'FILLED'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                else:
                    if i + X+1 > last_index:
                        exit_order['status_exit'][i] = 'OPEN'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    else:
                        exit_order['status_exit'][i] = 'TIMEOUT'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
            elif exit_order['ticker'][i].values[0] == 'ko' and exit_order['action'][i].values[0] == 'sell':
                for j in range(i + 1, i + X + 1):
                    Loss = -((prices_dataframe['ko_Close'][j] - exit_order['price'][i].values[0]) /
                             exit_order['price'][i].values[0]) * 100
                    if Loss > L:  # L=20
                        exit_order['status_exit'][i] = 'STOPLOSS'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    elif exit_order['price'][i].values[0] < prices_dataframe['ko_Close'][j]:
                        exit_order['status_exit'][i] = 'FILLED'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    else:
                        if i + X+1 > last_index:
                            exit_order['status_exit'][i] = 'OPEN'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
                        else:
                            exit_order['status_exit'][i] = 'TIMEOUT'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
            elif exit_order['ticker'][i].values[0] == 'pep' and exit_order['action'][i].values[0] == 'buy':
                for j in range(i + 1, i + X + 1):
                    Loss = ((prices_dataframe['pep_Close'][j] - exit_order['price'][i].values[0]) /
                            exit_order['price'][i].values[0]) * 100
                    if Loss > L:  # L=20
                        exit_order['status_exit'][i] = 'STOPLOSS'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    elif exit_order['price'][i].values[0] > prices_dataframe['pep_Close'][j + 1]:
                        exit_order['status_exit'][i] = 'FILLED'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    else:
                        if i + X + 1 > last_index:
                            exit_order['status_exit'][i] = 'OPEN'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
                        else:
                            exit_order['status_exit'][i] = 'TIMEOUT'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
            else:
                for j in range(i + 1, i + X + 1):
                    Loss = -((prices_dataframe['pep_Close'][j] - exit_order['price'][i].values[0]) /
                             exit_order['price'][i].values[0]) * 100
                    if Loss > L:  # L=20
                        exit_order['status_exit'][i] = 'STOPLOSS'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    elif exit_order['price'][i].values[0] < prices_dataframe['pep_Close'][j + 1]:
                        exit_order['status_exit'][i] = 'FILLED'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    else:
                        if i + X + 1 > last_index:
                            exit_order['status_exit'][i] = 'OPEN'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
                        else:
                            exit_order['status_exit'][i] = 'TIMEOUT'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
        else:
            continue

    # format the dataframe
    del exit_order['date'], exit_order['status']
    order = ['date_exit', 'ticker', 'price', 'quantity', 'action', 'trip', 'status_exit']
    exit_order = exit_order[order]
    exit_order.columns = ['date', 'ticker', 'price', 'quantity', 'action', 'trip', 'status']
    exit_order['date'] = pd.to_datetime(exit_order['date'])

    return exit_order
    # concat entry_order and exit_order
    # frames = [exit_order, entry_trade]
    # result = pd.concat(frames)
    # result['date'] = pd.to_datetime(result['date'])
    # result.sort_values(by='date', inplace=True, ascending=True)
    # return result

In [59]:
csv_file = "worriors_data_raw.csv"
csv_data = pd.read_csv(csv_file, low_memory=False)  # 防止弹出警告
csv_df = pd.DataFrame(csv_data)
del csv_df['Unnamed: 9'], csv_df['Unnamed: 10'], csv_df['Unnamed: 11'], csv_df['Unnamed: 12']
result = enter_trade(60, csv_df, 1)
exit_result = exit_trade(60, csv_df, result, 0.02, 20, 60)

/var/folders/q5/8pth12v54gs74jcrx3l_hx540000gn/T/ipykernel_76649/2184084346.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_y.rename(columns={'ticker_y': 'ticker', 'price_y': 'price', 'amount_y': 'quantity', 'action_y': 'action'},
/var/folders/q5/8pth12v54gs74jcrx3l_hx540000gn/T/ipykernel_76649/2184084346.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x.rename(columns={'ticker_x': 'ticker', 'price_x': 'price', 'amount_x': 'quantity', 'action_x': 'action'},
/var/folders/q5/8pth12v54gs74jcrx3l_hx540000gn/T/ipykernel_76649/1033909671.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [60]:
exit_result

,date,ticker,price,quantity,action,trip,status
83,2017-05-05,ko,44.2884,100.000000,buy,exit,FILLED
83,2017-05-05,pep,109.9266,38.709102,sell,exit,FILLED
84,2017-05-08,pep,110.7792,38.738500,sell,exit,FILLED
84,2017-05-08,ko,44.6658,100.000000,buy,exit,FILLED
85,2017-05-09,ko,44.4618,100.000000,buy,exit,FILLED
...,...,...,...,...,...,...,...
1309,2022-03-18,ko,60.6492,100.000000,buy,exit,FILLED
1310,2022-03-21,pep,158.1524,37.365225,sell,exit,FILLED
1310,2022-03-21,ko,61.5060,100.000000,buy,exit,FILLED
1312,2022-03-23,ko,62.1486,100.000000,buy,exit,FILLED


In [61]:
exit_order_mkt = exit_result.copy(deep=True)

# select rows where status is TIMEOUT or STOPLOSS
# exit_order_mkt = exit_order_mkt.loc[exit_order_mkt['status'].isin(['STOPLOSS', 'TIMEOUT'])]

exit_order_mkt

,date,ticker,price,quantity,action,trip,status
83,2017-05-05,ko,44.2884,100.000000,buy,exit,FILLED
83,2017-05-05,pep,109.9266,38.709102,sell,exit,FILLED
84,2017-05-08,pep,110.7792,38.738500,sell,exit,FILLED
84,2017-05-08,ko,44.6658,100.000000,buy,exit,FILLED
85,2017-05-09,ko,44.4618,100.000000,buy,exit,FILLED
...,...,...,...,...,...,...,...
1309,2022-03-18,ko,60.6492,100.000000,buy,exit,FILLED
1310,2022-03-21,pep,158.1524,37.365225,sell,exit,FILLED
1310,2022-03-21,ko,61.5060,100.000000,buy,exit,FILLED
1312,2022-03-23,ko,62.1486,100.000000,buy,exit,FILLED


In [62]:
first_index = exit_order_mkt.first_valid_index()
last_index = exit_order_mkt.index[-1]

print(first_index)
print(last_index)

83
1312


In [63]:
exit_order_mkt['price_mkt'] = exit_order_mkt['price']
# exit_order_mkt['status_mkt'] = 'FILLED'

In [64]:
for i in range(first_index, last_index + 1):
    if i in exit_order_mkt.index:
        if exit_order_mkt['status'][i].values[0] == 'STOPLOSS' or exit_order_mkt['status'][i].values[1] == 'STOPLOSS' or exit_order_mkt['status'][i].values[0] == 'TIMEOUT' or exit_order_mkt['status'][i].values[1] == 'TIMEOUT':
            if exit_order_mkt['ticker'][i].values[0] == 'ko':
                exit_order_mkt['price_mkt'][i].values[0] = csv_df['ko_Close'][i + 2]
                exit_order_mkt['price_mkt'][i].values[1] = csv_df['pep_Close'][i + 2]
            else:
                exit_order_mkt['price_mkt'][i].values[0] = csv_df['pep_Close'][i + 2]
                exit_order_mkt['price_mkt'][i].values[1] = csv_df['ko_Close'][i + 2]
        else:
            exit_order_mkt['price_mkt'][i].values[0] = exit_order_mkt['price'][i].values[0]
            exit_order_mkt['price_mkt'][i].values[1] = exit_order_mkt['price'][i].values[1]
    else:
        continue

In [65]:
exit_order_mkt

,date,ticker,price,quantity,action,trip,status,price_mkt
83,2017-05-05,ko,44.2884,100.000000,buy,exit,FILLED,44.2884
83,2017-05-05,pep,109.9266,38.709102,sell,exit,FILLED,109.9266
84,2017-05-08,pep,110.7792,38.738500,sell,exit,FILLED,110.7792
84,2017-05-08,ko,44.6658,100.000000,buy,exit,FILLED,44.6658
85,2017-05-09,ko,44.4618,100.000000,buy,exit,FILLED,44.4618
...,...,...,...,...,...,...,...,...
1309,2022-03-18,ko,60.6492,100.000000,buy,exit,FILLED,60.6492
1310,2022-03-21,pep,158.1524,37.365225,sell,exit,FILLED,158.1524
1310,2022-03-21,ko,61.5060,100.000000,buy,exit,FILLED,61.5060
1312,2022-03-23,ko,62.1486,100.000000,buy,exit,FILLED,62.1486


In [66]:
# format the dataframe
del exit_order_mkt['price']
order2 = ['date', 'ticker', 'price_mkt', 'quantity', 'action', 'trip', 'status']
exit_order_mkt = exit_order_mkt[order2]
exit_order_mkt.columns = ['date', 'ticker', 'price', 'quantity', 'action', 'trip', 'status']
exit_order_mkt['date'] = pd.to_datetime(exit_order_mkt['date'])

exit_order_mkt

,date,ticker,price,quantity,action,trip,status
83,2017-05-05,ko,44.2884,100.000000,buy,exit,FILLED
83,2017-05-05,pep,109.9266,38.709102,sell,exit,FILLED
84,2017-05-08,pep,110.7792,38.738500,sell,exit,FILLED
84,2017-05-08,ko,44.6658,100.000000,buy,exit,FILLED
85,2017-05-09,ko,44.4618,100.000000,buy,exit,FILLED
...,...,...,...,...,...,...,...
1309,2022-03-18,ko,60.6492,100.000000,buy,exit,FILLED
1310,2022-03-21,pep,158.1524,37.365225,sell,exit,FILLED
1310,2022-03-21,ko,61.5060,100.000000,buy,exit,FILLED
1312,2022-03-23,ko,62.1486,100.000000,buy,exit,FILLED


In [67]:
# frames = [exit_result, exit_order_mkt]
# result = pd.concat(frames)
exit_order_mkt['date'] = pd.to_datetime(exit_order_mkt['date'])
exit_order_mkt.sort_values(by='date', inplace=True, ascending=False)

exit_order_mkt

,date,ticker,price,quantity,action,trip,status
1312,2022-03-23,pep,159.5832,37.417097,sell,exit,FILLED
1312,2022-03-23,ko,62.1486,100.000000,buy,exit,FILLED
1310,2022-03-21,ko,61.5060,100.000000,buy,exit,FILLED
1310,2022-03-21,pep,158.1524,37.365225,sell,exit,FILLED
1309,2022-03-18,ko,60.6492,100.000000,buy,exit,FILLED
...,...,...,...,...,...,...,...
85,2017-05-09,pep,110.9360,38.507067,sell,exit,FILLED
84,2017-05-08,pep,110.7792,38.738500,sell,exit,FILLED
84,2017-05-08,ko,44.6658,100.000000,buy,exit,FILLED
83,2017-05-05,pep,109.9266,38.709102,sell,exit,FILLED
